In [143]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

In [3]:
df = spark.read.parquet("data/unidos/merge_jugadores.parquet")
df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Born: float (nullable = true)
 |-- MP: float (nullable = true)
 |-- Starts: float (nullable = true)
 |-- Min: float (nullable = true)
 |-- Gls: float (nullable = true)
 |-- Ast: float (nullable = true)
 |-- YellowC: float (nullable = true)
 |-- RedC: float (nullable = true)
 |-- PrgC: float (nullable = true)
 |-- PrgP: float (nullable = true)
 |-- PrgR: float (nullable = true)
 |-- Fls: float (nullable = true)
 |-- Off: float (nullable = true)
 |-- Crosses: float (nullable = true)
 |-- Recov: float (nullable = true)
 |-- Aerialwon%: float (nullable = true)
 |-- Tack_Def_3rd: float (nullable = true)
 |-- Tack_Mid_3rd: float (nullable = true)
 |-- Tack_Att_3rd: float (nullable = true)
 |-- Tkl%: float (nullable = true)
 |-- Tkl: float (nullable = true)
 |-- TklW: float (nullable = true)
 |-- Int: float (nullable = true)
 |--

In [141]:
sources = {
        "stats": spark.read.option("header", True).csv("data/limpios/stats/stats.csv"),
        "misc": spark.read.option("header", True).csv("data/limpios/misc/misc.csv"),
        "defense": spark.read.option("header", True).csv("data/limpios/defense/defense.csv"),
        "passing": spark.read.option("header", True).csv("data/limpios/passing/passing.csv"),
        "possession": spark.read.option("header", True).csv("data/limpios/possession/possession.csv"),
        "shooting": spark.read.option("header", True).csv("data/limpios/shooting/shooting.csv"),
        "mercado": spark.read.option("header", True).csv("data/limpios/mercado/mercado.csv"),
        "keepers": spark.read.option("header", True).csv("data/limpios/keepers/keepers.csv"),
        "keepersadv": spark.read.option("header", True).csv("data/limpios/keepersadv/keepersadv.csv")
    }

unido2 = sources["stats"] \
        .join(sources["misc"], ["Player", "Squad", "Season", "Competition"]) \
        .join(sources["defense"], ["Player", "Squad", "Season", "Competition"]) \
        .join(sources["passing"], ["Player", "Squad", "Season", "Competition"]) \
        .join(sources["possession"], ["Player", "Squad", "Season", "Competition"]) \
        .join(sources["shooting"], ["Player", "Squad", "Season", "Competition"]) \
        .join(sources["keepers"], ["Player", "Squad", "Season", "Competition"], "left") \
        .join(sources["keepersadv"], ["Player", "Squad", "Season", "Competition"], "left")

In [ ]:
from pathlib import Path
import pandas as pd

sources = {
        "stats": spark.read.option("header", True).csv("data/v1/stats.csv"),
        "misc": spark.read.option("header", True).csv("data/v1/misc.csv"),
        "defense": spark.read.option("header", True).csv("data/v1/defense.csv"),
        "passing": spark.read.option("header", True).csv("data/v1/passing.csv"),
        "possession": spark.read.option("header", True).csv("data/v1/possession.csv"),
        "shooting": spark.read.option("header", True).csv("data/v1/shooting.csv"),
        "mercado": spark.read.option("header", True).csv("data/v1/mercado.csv"),
        "keepers": spark.read.option("header", True).csv("data/v1/keepers.csv"),
        "keepersadv": spark.read.option("header", True).csv("data/v1/keepersadv.csv")
    }


unido = sources["stats"] \
        .join(sources["misc"], ["Player", "Squad", "Season", "Liga"]) \
        .join(sources["defense"], ["Player", "Squad", "Season", "Liga"]) \
        .join(sources["passing"], ["Player", "Squad", "Season", "Liga"]) \
        .join(sources["possession"], ["Player", "Squad", "Season", "Liga"]) \
        .join(sources["shooting"], ["Player", "Squad", "Season", "Liga"]) \
        .join(sources["keepers"], ["Player", "Squad", "Season", "Liga"], "left") \
        .join(sources["keepersadv"], ["Player", "Squad", "Season", "Liga"], "left")



In [140]:
unido.count()

2854

In [142]:
unido2.filter(col("Season") == "2024-2025").count()

2854

In [78]:
unido.show(1, truncate=False)

+------+-----+------+----+----+---+------+---+---+---+----+----+----+----+----+-------+-------+-------+----+-----+----+---+------+---+----+---+---+---+---+---+-----+----+---+----+-----+------+-----+------+-----+------+---+---+-----+---+---+-----+-------+-------+-------+-------+----+---+-----+----+-----+-------+---+---+---+---+---+---+----+---+----+-------+----+---+----+-----+---+-----+-------+----+--------+-------+-------+---+----+----+----+---+------+-----+
|Player|Squad|Season|Liga|Born|MP |Starts|Min|Gls|Ast|CrdY|CrdR|PrgC|PrgP|PrgR|Def 3rd|Mid 3rd|Att 3rd|Tkl%|Tkl.1|TklW|Int|Blocks|Sh |Pass|Clr|Err|Fls|Off|Crs|Recov|Won%|Cmp|Cmp%|Att.1|Cmp%.1|Att.2|Cmp%.2|Att.3|Cmp%.3|xAG|xA |A-xAG|1/3|PPA|CrsPA|Def 3rd|Mid 3rd|Att 3rd|Att Pen|Live|Att|Succ%|Tkld|Tkld%|PrgDist|1/3|CPA|Mis|Dis|Sh |SoT|G/Sh|xG |npxG|npxG/Sh|G-xG|GA |SoTA|Save%|CS%|PKatt|Save%.1|PSxG|PSxG/SoT|PSxG+/-|Launch%|Opp|Stp%|#OPA|None|Pos|Height|Value|
+------+-----+------+----+----+---+------+---+---+---+----+----+----+----+

In [54]:

unido.write.mode("overwrite").parquet("data copy/unidos.parquet")

AnalysisException: [COLUMN_ALREADY_EXISTS] The column `1/3` already exists. Choose another name or rename the existing column. SQLSTATE: 42711

In [45]:
unidos = spark.read.csv("data copy/uniddos")

In [50]:
unidos.columns  # cambia df2 por tu DF

[]

In [ ]:
from pyspark.sql.functions import col



df_union = df2.toPandas()

df_union.to_csv("prueba_data/performance.csv", index=False)

In [10]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("adjusted_score").isNotNull())&(col("Min") > 900))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("Evaluated_Position").orderBy(col("adjusted_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") == 1) \
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df1).select("Player", "Pos", "Squad", "Competition", "performance_score").show()


+--------------------+------------------+--------------+--------------+-------------------+
|              Player|               Pos|         Squad|   Competition|  performance_score|
+--------------------+------------------+--------------+--------------+-------------------+
|     bruno fernandes|Attacking Midfield|manchester utd|premier-league| 0.4600860900381374|
|               pedri|  Central Midfield|     barcelona|        laliga| 0.4287534923294778|
|         daley blind|       Centre-Back|        girona|        laliga| 0.3261812019322885|
|       kylian mbappe|    Centre-Forward|   real madrid|        laliga| 0.2146943242814554|
|      joshua kimmich|Defensive Midfield| bayern munich|    bundesliga| 0.5666147573187089|
|       robin zentner|        Goalkeeper|      mainz 05|    bundesliga| 0.4554211702517768|
|    marcus tavernier|     Left Midfield|   bournemouth|premier-league| 0.5959664719230389|
|            raphinha|       Left Winger|     barcelona|        laliga|0.3059874

In [11]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("adjusted_score").isNotNull())&(col("Min") > 900))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("Evaluated_Position").orderBy(col("adjusted_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") == 1) \
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df2).select("Player", "Pos", "Squad", "Competition", "performance_score").show()

+--------------------+------------------+--------------+--------------+-------------------+
|              Player|               Pos|         Squad|   Competition|  performance_score|
+--------------------+------------------+--------------+--------------+-------------------+
|     bruno fernandes|Attacking Midfield|manchester utd|premier-league| 0.4600860900381374|
|               pedri|  Central Midfield|     barcelona|        laliga| 0.4287534923294778|
|         daley blind|       Centre-Back|        girona|        laliga| 0.3261812019322885|
|       kylian mbappe|    Centre-Forward|   real madrid|        laliga| 0.2146943242814554|
|      joshua kimmich|Defensive Midfield| bayern munich|    bundesliga| 0.5666147573187089|
|       robin zentner|        Goalkeeper|      mainz 05|    bundesliga| 0.4554211702517768|
|    marcus tavernier|     Left Midfield|   bournemouth|premier-league| 0.5959664719230389|
|            raphinha|       Left Winger|     barcelona|        laliga|0.3059874

In [27]:
df1.filter(col("Player") == "thibaut courtois").show()

+----------------+-----------+---------+-----------+----------+------+-----+------------------+-------------+------------------+------------------+
|          Player|      Squad|   Season|Competition|       Pos|   Min|Value| performance_score|penalty_score|    adjusted_score|Evaluated_Position|
+----------------+-----------+---------+-----------+----------+------+-----+------------------+-------------+------------------+------------------+
|thibaut courtois|real madrid|2023-2024|     laliga|Goalkeeper| 332.0|2.8E7|0.3738839432952276|          0.0|0.3738839432952276|        Goalkeeper|
|thibaut courtois|real madrid|2024-2025|     laliga|Goalkeeper|2700.0|2.0E7|0.3363258408796188|       0.0015|0.3348258408796188|        Goalkeeper|
|thibaut courtois|real madrid|2022-2023|     laliga|Goalkeeper|2790.0|4.5E7|0.4346668970958383|       0.0021|0.4325668970958383|        Goalkeeper|
+----------------+-----------+---------+-----------+----------+------+-----+------------------+-------------+---

In [27]:
from pyspark.sql.functions import col, desc

# Filtro mínimo de minutos por posición (ajustable)

def top_10_por_posicion_temporada(df, posicion, temporada, variable_score):
    min_minutos = 900

    return (
        df.filter((col("Season") == temporada) & (col("Min") >= min_minutos)&(col("Pos") == posicion))
          .orderBy(desc(variable_score))
          .limit(10)
    )
top_10_por_posicion_temporada(df1, "Centre-Forward", "2024-2025", "performance_score").show()

+-------------------+---------------+---------+--------------+--------------+------+---------+-------------------+--------------------+-------------------+------------------+
|             Player|          Squad|   Season|   Competition|           Pos|   Min|    Value|  performance_score|       penalty_score|     adjusted_score|Evaluated_Position|
+-------------------+---------------+---------+--------------+--------------+------+---------+-------------------+--------------------+-------------------+------------------+
|      kylian mbappe|    real madrid|2024-2025|        laliga|Centre-Forward|2907.0|    1.8E8| 0.2146943242814554|              0.0928|0.12189432428145541|    Centre-Forward|
|benedict hollerbach|   union berlin|2024-2025|    bundesliga|Centre-Forward|2540.0|    1.2E7|0.15549765424037695|              0.1348|0.02069765424037695|    Centre-Forward|
|  antoine griezmann|atlético madrid|2024-2025|        laliga|Centre-Forward|2469.0|    1.8E7|0.15420179685048288|0.045700000

In [41]:
from pyspark.sql.functions import col, desc

# Filtro mínimo de minutos por posición (ajustable)

def top_10_por_posicion_temporada(df, posicion, temporada, variable_score):
    min_minutos = 900

    return (
        df.filter((col("Season") == temporada) & (col("Min") >= min_minutos)&(col("Pos") == posicion))
          .orderBy(desc(variable_score))
          .limit(10)
    )
top_10_por_posicion_temporada(df2, "Right Winger", "2024-2025", "performance_score").show()

+-----------------+-------------+---------+--------------+------------+------+----+------+-----+-------------------+-------------------+-------------------+------------------+
|           Player|        Squad|   Season|   Competition|         Pos|   Min|  MP|Starts|Value|  performance_score|      penalty_score|     adjusted_score|Evaluated_Position|
+-----------------+-------------+---------+--------------+------------+------+----+------+-----+-------------------+-------------------+-------------------+------------------+
|     lamine yamal|    barcelona|2024-2025|        laliga|Right Winger|2856.0|35.0|  31.0|2.0E8|0.47175166104512783|             0.0833|0.38845166104512785|      Right Winger|
|  ousmane dembele|    paris s-g|2024-2025|       ligue-1|Right Winger|1730.0|29.0|  20.0|9.0E7| 0.4639409450492875|0.07210000000000001| 0.3918409450492875|      Right Winger|
|    michael olise|bayern munich|2024-2025|    bundesliga|Right Winger|2334.0|34.0|  28.0|1.0E8| 0.4470492376441892|    

In [42]:
from pyspark.sql.functions import col, desc

# Filtro mínimo de minutos por posición (ajustable)

def top_10_por_posicion_temporada(df, posicion, temporada, variable_score):
    min_minutos = 900

    return (
        df.filter((col("Pos") == posicion) & (col("Season") == temporada) & (col("Min") >= min_minutos))
          .orderBy(desc(variable_score))
          .limit(10)
    )
top_10_por_posicion_temporada(df1, "Right Winger", "2024-2025", "adjusted_score").show()


+-------------+-------------+---------+--------------+------------+------+-----+-------------------+--------------------+-------------------+------------------+
|       Player|        Squad|   Season|   Competition|         Pos|   Min|Value|  performance_score|       penalty_score|     adjusted_score|Evaluated_Position|
+-------------+-------------+---------+--------------+------------+------+-----+-------------------+--------------------+-------------------+------------------+
| lamine yamal|    barcelona|2024-2025|        laliga|Right Winger|2856.0|2.0E8| 0.5085580010780223|              0.0806| 0.4279580010780223|      Right Winger|
|michael olise|bayern munich|2024-2025|    bundesliga|Right Winger|2334.0|1.0E8| 0.4847074106859111|              0.0615| 0.4232074106859111|      Right Winger|
|mohamed salah|    liverpool|2024-2025|premier-league|Right Winger|3371.0|5.0E7| 0.4199367656254368|              0.0831| 0.3368367656254368|      Right Winger|
|  bukayo saka|      arsenal|2024-

In [18]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("performance_score").isNotNull()))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("Evaluated_Position").orderBy(col("performance_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") == 1) \
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df1).select("Player", "Pos", "Squad", "Competition", "adjusted_score", "performance_score", "penalty_score").show()

ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

In [9]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, min, max, when

def ajustar_penalty_score(df):
    # Ventana por posición y temporada
    w = Window.partitionBy("Evaluated_Position", "Season")

    # Calcular mínimo y máximo por grupo
    df = df.withColumn("penalty_min", min("penalty_score").over(w)) \
           .withColumn("penalty_max", max("penalty_score").over(w))

    # Normalizar penalty_score a [0, 1]
    df = df.withColumn(
        "penalty_score_scaled",
        when(
            col("penalty_max") != col("penalty_min"),
            (col("penalty_score") - col("penalty_min")) / (col("penalty_max") - col("penalty_min"))
        ).otherwise(0.0)
    )

    # Ajustar performance_score
    df = df.withColumn(
        "adjusted_score",
        col("performance_score") - col("penalty_score_scaled")
    )

    # Limpiar columnas auxiliares si no las necesitas
    df = df.drop("penalty_min", "penalty_max")

    return df

df2 = ajustar_penalty_score(df1)

In [21]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("adjusted_score").isNotNull()))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("Evaluated_Position").orderBy(col("performance_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") == 2) \
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df2).select("Player", "Pos", "Squad", "Competition", "adjusted_score", "performance_score", "penalty_score").show()

+----------------+------------------+---------------+--------------+-------------------+-------------------+--------------------+
|          Player|               Pos|          Squad|   Competition|     adjusted_score|  performance_score|       penalty_score|
+----------------+------------------+---------------+--------------+-------------------+-------------------+--------------------+
| bruno fernandes|Attacking Midfield| manchester utd|premier-league|0.36136104251615236|0.42546104251615235|              0.0641|
|         vitinha|  Central Midfield|      paris s-g|       ligue-1|0.33007409718788255|0.36977409718788257|              0.0397|
|      mario gila|       Centre-Back|          lazio|       serie-a|0.27763304168832925| 0.3126330416883293|               0.035|
|   tammy abraham|    Centre-Forward|           roma|       serie-a|-0.6395062725298684|0.26049372747013166|                 0.9|
|manuel locatelli|Defensive Midfield|       juventus|       serie-a|0.39621294102683824| 0